In [ ]:
import joblib
import rasterio
import rasterio.plot
import matplotlib.pyplot
import sklearn
import sklearn.ensemble
import pandas
import numpy

import analysis
import data
import datasets
import satellite

In [ ]:
AGGREGATE_LAYER_COUNT = 3
AGGREGATE_LAYER_START = 1
GRID_SEARCH_PIXEL_STEP_X = 16
GRID_SEARCH_PIXEL_STEP_Y = 16
MEASUREMENT_DISAMBIGUATION_MODE = lambda series: series.mean([TARGET_COLUMN])
PIXEL_LAYER_COUNT = 3
PIXEL_LAYER_START = 0
POSITIVE_THRESHOLD = 1000
SEED = 3617
# TARGET_COLUMN = "LI"
TARGET_COLUMN = "Li_ppm"

In [ ]:
satellite_image = datasets.satellite_image_2023()
# satellite_image = datasets.satellite_image_2019()

In [ ]:
satellite_data_frame = datasets.geochemical_satellite_analysis_2023_12_19(satellite_image, TARGET_COLUMN, POSITIVE_THRESHOLD)
# satellite_data_frame = datasets.geochemical_satellite_analysis_2023_10_17(satellite_image, TARGET_COLUMN, POSITIVE_THRESHOLD)
# satellite_data_frame = datasets.geochemical_satellite_analysis_2023_01_20(satellite_image, TARGET_COLUMN, POSITIVE_THRESHOLD)
satellite.create_features(satellite_data_frame, satellite_image, PIXEL_LAYER_START, PIXEL_LAYER_COUNT, AGGREGATE_LAYER_START, AGGREGATE_LAYER_COUNT)

In [ ]:
index_data_frame = satellite.create_summary_data_frame(satellite_data_frame, satellite_image, TARGET_COLUMN, MEASUREMENT_DISAMBIGUATION_MODE, PIXEL_LAYER_START, PIXEL_LAYER_COUNT, AGGREGATE_LAYER_START, AGGREGATE_LAYER_COUNT)
x_labels = index_data_frame.x_labels()
y_labels = index_data_frame.y_labels()

In [ ]:
#380 0,0,0,1
#84 0,1,1,5
#119 0,0,0,3
#48 0,0,0,7
#MEAN 52 0,3,1,3
#MAX 226 0,3,1,3 0.309859

x_train, y_train, x_test, y_test = data.split_train_test(index_data_frame, x_labels, y_labels, seed=SEED)

histogram = sklearn.ensemble.HistGradientBoostingRegressor(loss="squared_error", validation_fraction=0.2, min_samples_leaf=5, random_state=SEED).fit(x_train, y_train)

print("Train")
print("Training Score", histogram.score(x_train, y_train))
print("Test Score", histogram.score(x_test, y_test))

y_test_predicted = histogram.predict(x_test)
comparison, confusion_matrix, geochem_result_set = analysis.evaluate_performance(y_test, y_test_predicted, POSITIVE_THRESHOLD)

x_all = index_data_frame[x_labels].to_numpy(numpy.float32)
y_all = index_data_frame[y_labels].to_numpy(numpy.float32).ravel()
y_all_predicted = histogram.predict(x_all)

print("\nAll")
comparison_all, confusion_matrix_all, geochem_result_set_all = analysis.evaluate_performance(y_all, y_all_predicted, POSITIVE_THRESHOLD)

import joblib
joblib.dump(histogram, f"Models/Satellite{TARGET_COLUMN}Histogram.joblib")

# import skl2onnx 
# import skl2onnx.common.data_types
# initial_type = [("input", skl2onnx.common.data_types.FloatTensorType([None, x_train.shape[1]]))]
# with open(f"Models/Satellite{TARGET_COLUMN}Histogram.onnx", "wb") as file:
#     file.write(skl2onnx.convert_sklearn(histogram, initial_types=initial_type).SerializeToString())

In [ ]:
satellite_data_frame[f"{TARGET_COLUMN}_PREDICTED"] = histogram.predict(satellite_data_frame[x_labels])
satellite_data_frame.to_csv(f"Output/{TARGET_COLUMN} Histogram Satellite Predictions.csv", sep=",", index=False)

In [ ]:
grid_search_data_frame = satellite.create_grid_search_data_frame(satellite_image, GRID_SEARCH_PIXEL_STEP_X, GRID_SEARCH_PIXEL_STEP_Y, PIXEL_LAYER_START, PIXEL_LAYER_COUNT, AGGREGATE_LAYER_START, AGGREGATE_LAYER_COUNT)

In [ ]:
grid_search_data_frame = satellite.filter_white_pixels(grid_search_data_frame)

In [ ]:
grid_search_data_frame[TARGET_COLUMN] = histogram.predict(grid_search_data_frame[x_labels])
grid_search_data_frame[f"{TARGET_COLUMN}_THRESHOLD"] = grid_search_data_frame[TARGET_COLUMN].ge(POSITIVE_THRESHOLD)
grid_search_data_frame.to_csv(f"Output/{TARGET_COLUMN} Histogram Grid Search.csv", sep=",", index=False)

In [ ]:
joblib.dump(grid_search_data_frame, f"Output/{TARGET_COLUMN} Grid Search Data Frame.joblib")

In [ ]:
abridged_grid_search_data_frame = grid_search_data_frame[["EAST", "NORTH", TARGET_COLUMN]]
abridged_grid_search_data_frame.to_csv(f"Output/Abridged {TARGET_COLUMN} Histogram Grid Search.csv", sep=",", index=False)

In [ ]:
previous_model = joblib.load(f"Models/{TARGET_COLUMN}_HistogramGradientBoostingRegressor.joblib")

# Newest geochem
ratios = ["Rb_K2O", "Rb_Yb", "Rb_La", "K2O_MgO", "Rb_MgO", "U_Ba", "Ba_La", "U_Th", "Rb_Sn", "K2O_Sn", "Rb_MnO", "MnO_MgO"]
x_geochemical_labels = ["SIO2_wt%", "TIO2_wt%", "AL2O3_wt%", "FE2O3T_wt%", "FEOT_wt%", "MNO_wt%", "MGO_wt%", "CAO_wt%", "NA2O_wt%", "K2O_wt%", "P2O5_wt%", "H2O_wt%", "CR2O3_wt%"] + ["Li_ppm", "Sc_ppm", "V_ppm", "Cr_ppm", "Co_ppm", "Ni_ppm", "Cu_ppm", "Zn_ppm", "Ga_ppm", "Mo_ppm", "W_ppm", "Sn_ppm", "Sb_ppm", "Rb_ppm", "Sr_ppm", "Y_ppm", "Nb_ppm", "Zr_ppm", "Cs_ppm", "Cd_ppm", "Ba_ppm", "La_ppm", "Ce_ppm", "Pr_ppm", "Nd_ppm", "Sm_ppm", "Eu_ppm", "Gd_ppm", "Tb_ppm", "Dy_ppm", "Ho_ppm", "Er_ppm", "Tm_ppm", "Yb_ppm", "Lu_ppm", "Hf_ppm", "Ta_ppm", "Pb_ppm", "Th_ppm", "U_ppm", "Au_ppb", "Ag_ppm", "S_%", "As_ppm", "Se_ppm", "Te_ppm", "Ge_ppm", "Bi_ppm", "Tl_ppm", "Be_ppm", "B_ppm", "F_ppm", "Cl_ppm", "In_ppm"] + ratios
x_geochemical_labels.remove(TARGET_COLUMN)
datasets.compute_ratios(satellite_data_frame, ratios, x_geochemical_labels + [TARGET_COLUMN])

# Previous geochem
# datasets.compute_ratios(satellite_data_frame)
# x_geochemical_labels = ["SIO2", "TIO2", "AL2O3","FE2O3T","FEOT","MNO","MGO","CAO","NA2O","K2O","P2O5","H2O", "CR2O3"] + ["LI", "SC", "V","CR","CO","NI","CU","ZN","GA","MO","W","SN","SB","RB","SR","Y","NB","ZR","CS","CD","BA","LA","CE","PR","ND","SM","EU","GD","TB","DY","HO","ER","TM","YB","LU","HF","TA","PB","TH","U","AU","AG","S","AS_","SE","TE","GE","BI","TL","BE","B","F","CL","INDIUM"] + ["Rb_K2O", "Rb_Yb", "Rb_La", "K2O_MgO", "Rb_MgO", "U_Ba", "Ba_La", "U_Th", "Rb_Sn", "K2O_Sn", "Rb_MnO", "MnO_MgO"]
# x_geochemical_labels.remove(TARGET_COLUMN)

satellite_data_frame[f"{TARGET_COLUMN}_PREDICTED_GEOCHEMICAL"] = previous_model.predict(satellite_data_frame[x_geochemical_labels])
satellite_data_frame[f"{TARGET_COLUMN}_PREDICTED_GEOCHEMICAL_THRESHOLD"] = satellite_data_frame[f"{TARGET_COLUMN}_PREDICTED_GEOCHEMICAL"].ge(POSITIVE_THRESHOLD)

In [ ]:
figure, axes = matplotlib.pyplot.subplots(nrows=1, ncols=3, figsize=(20, 10))
satellite_data_frame[satellite_data_frame[f"{TARGET_COLUMN}_THRESHOLD"] == False].plot(ax=axes[0], legend=True, color="white", markersize=9, linewidth=0.5, edgecolors="black")

satellite_data_frame[satellite_data_frame[f"{TARGET_COLUMN}_THRESHOLD"]].plot(ax=axes[0], legend=True, color="yellow", markersize=25, linewidth=0.5, edgecolors="black")
axes[0].set_title(f"Known Samples")

# grid_search_data_frame[grid_search_data_frame[f"{TARGET_COLUMN}_THRESHOLD"]].sort_values(f"{TARGET_COLUMN}", ascending=False).head(14805).plot(ax=axes[1], legend=True, color="yellow", markersize=1)
# grid_search_data_frame[grid_search_data_frame[f"{TARGET_COLUMN}_THRESHOLD"]].sort_values(f"{TARGET_COLUMN}", ascending=False).head(11103).plot(ax=axes[1], legend=True, color="yellow", markersize=1)
# grid_search_data_frame[grid_search_data_frame[f"{TARGET_COLUMN}_THRESHOLD"]].sort_values(f"{TARGET_COLUMN}", ascending=False).head(7402).plot(ax=axes[1], legend=True, color="yellow", markersize=1)
# grid_search_data_frame[grid_search_data_frame[f"{TARGET_COLUMN}_THRESHOLD"]].sort_values(f"{TARGET_COLUMN}", ascending=False).head(3701).plot(ax=axes[1], legend=True, color="yellow", markersize=1)
# grid_search_data_frame[grid_search_data_frame[f"{TARGET_COLUMN}_THRESHOLD"]].sort_values(f"{TARGET_COLUMN}", ascending=False).head(1480).plot(ax=axes[1], legend=True, color="yellow", markersize=1)
# grid_search_data_frame[grid_search_data_frame[f"{TARGET_COLUMN}_THRESHOLD"]].sort_values(f"{TARGET_COLUMN}", ascending=False).head(740).plot(ax=axes[1], legend=True, color="yellow", markersize=1)
grid_search_data_frame[grid_search_data_frame[f"{TARGET_COLUMN}_THRESHOLD"]].plot(ax=axes[1], legend=True, color="yellow", markersize=1)
axes[1].set_title(f"Grid Search Positive Results")

satellite_data_frame[satellite_data_frame[f"{TARGET_COLUMN}_PREDICTED_GEOCHEMICAL_THRESHOLD"]].plot(ax=axes[2], legend=True, color="yellow", markersize=25, linewidth=0.5, edgecolors="black")
axes[2].set_title(f"Predicted Positive Samples")

rasterio.plot.show(satellite_image, ax=axes[0])
rasterio.plot.show(satellite_image, ax=axes[1])
rasterio.plot.show(satellite_image, ax=axes[2])
matplotlib.pyplot.savefig(f"Output/{TARGET_COLUMN} Figure.png")

In [ ]:
plot = grid_search_data_frame[grid_search_data_frame[f"{TARGET_COLUMN}_THRESHOLD"]][f"{TARGET_COLUMN}"].plot(kind="hist", bins=200, figsize=(14, 4))

plot.set_title("Ppm distribution")
plot.set_xlabel("Li ppm")
plot.set_ylabel("frequency")

x = grid_search_data_frame[grid_search_data_frame[f"{TARGET_COLUMN}_THRESHOLD"]][f"{TARGET_COLUMN}"]

ymax = 0
for step in numpy.arange(0.05, 1, 0.05):
    quantile = round(step, 2)
    xval = x.quantile(quantile)
    label = f"{int(quantile * 100)}th"
    ymax += 80
    plot.vlines(xval, 0, ymax , linestyle='dashed', color="r", linewidth=1)
    plot.text(xval* 1.01, ymax, label.format(x.mean()))

x98th = x.quantile(0.98)
plot.vlines(x98th, 0, ymax , linestyle='dashed', color="r", linewidth=1)
plot.text(x98th* 1.01, ymax, '98th'.format(x98th))

x99th = x.quantile(0.99)
plot.vlines(x99th, 0, ymax , linestyle='dashed', color="r", linewidth=1)
plot.text(x99th* 1.01, ymax, '99th percentile'.format(x99th))

matplotlib.pyplot.savefig(f"Output/{TARGET_COLUMN} ppm frequency histogram.png")
